In [ ]:
import os
import sys

# Provjera pokreće li se kod na Google Colabu
if 'google.colab' in str(get_ipython()):
    print(">>> Pokretanje na Colabu. Priprema projekta s GitHuba...")
    
    GITHUB_USER = "lukaznj"
    REPO_NAME = "projekt-r"
    
    REPO_URL = f"https://github.com/{GITHUB_USER}/{REPO_NAME}.git"
    
    # 1. Kloniranje repozitorija ako već ne postoji
    if not os.path.exists(REPO_NAME):
        !git clone {REPO_URL}
    else:
        print(f">>> Repozitorij {REPO_NAME} već postoji.")

    # 2. Ulazak u mapu projekta
    %cd {REPO_NAME}
    
    # 3. Instalacija potrebnih biblioteka iz requirements.txt
    if os.path.exists("requirements.txt"):
        print(">>> Instalacija dependencyja...")
        !pip install -r requirements.txt
    else:
        print(">>> UPOZORENJE: requirements.txt nije pronađen!")

    # 4. Dodavanje trenutne mape u Python path (da import backend radi)
    sys.path.append(os.getcwd())
    
    print(">>> OKRUŽENJE SPREMNO!")
else:
    print(">>> Pokretanje lokalno (nije Colab). Provjerite jesu li datoteke u istoj mapi.")

# Provjera dostupnosti backend datoteke
if os.path.exists("backend.py"):
    import backend
    print(">>> Modul 'backend' uspješno uvezen.")
else:
    print(">>> KRITIČNA GREŠKA: 'backend.py' nije pronađen!")

# Rekonstrukcija RGB signala iz ne-Bayerova filtra u digitalnim kamerama

**Preddiplomski projekt - Tema 2**

Ovaj notebook implementira i evaluira metode rekonstrukcije RGB signala iz Quad Bayer i Nonacell mozaika. Za razliku od standardnog Bayerovog filtra, ovi mozaici grupiraju piksele iste boje u 2×2 (Quad Bayer) ili 3×3 (Nonacell) blokove, što omogućava fleksibilnu zamjenu između osjetljivosti i prostorne rezolucije.

## Dataset
Za evaluaciju metoda koristimo **Kodak Lossless True Color Image Suite**, koji je de facto standard u znanstvenoj zajednici za testiranje algoritama za obradu slike i demosaicing. Slike sadrže raznolike teksture, fine detalje i prijelaze boja koji su ključni za testiranje kvalitete rekonstrukcije.

## Pregled mozaika

- **Quad Bayer**: 2×2 blokovi jednake boje (4×4 super-piksel: R-R-G-G / R-R-G-G / G-G-B-B / G-G-B-B)
  - Za naš slučaj slimulacije, Quad Byer i Tetracell su zamijenjivi termini, 4 piksela se stapa u 1
- **Nonacell**: 3×3 blokovi jednake boje (6×6 super-piksel: R-R-R-G-G-G / R-R-R-G-G-G / R-R-R-G-G-G / G-G-G-B-B-B / G-G-G-B-B-B / G-G-G-B-B-B)
  - Karakterističan za senzore od 108 MP (npr. Samsung HM serija), gdje se 9 piksela stapa u 1

## Implementirane metode

1. **Direct reconstruction**: Izravna interpolacija koja eksplicitno poštuje blok strukturu
2. **Virtual Bayer**: Konverzija u standardni Bayer mozaik, zatim primjena klasičnih metoda
3. **Super-Resolution**: Hibridni pristup koji kombinira Direct i Virtual Bayer za maksimalnu rezoluciju

## Evaluacijske metrike

Za kvantificiranje kvalitete rekonstrukcije koristimo četiri ključne metrike. Evo kako ih interpretirati:

- **PSNR (Y) - Peak Signal-to-Noise Ratio (Luminancija)**
  - **Što mjeri:** Apsolutnu točnost rekonstrukcije svjetline (Y kanala). To je omjer signala i šuma u logaritamskoj skali.
  - **Interpretacija:** **Veće je bolje.** Vrijednosti >30 dB su dobre, >40 dB izvrsne. Niske vrijednosti ukazuju na vidljiv šum ili velika odstupanja.

- **SSIM (Y) - Structural Similarity Index (Luminancija)**
  - **Što mjeri:** Očuvanost strukture slike (bridovi, teksture), sličnije načinu na koji ljudsko oko vidi kvalitetu.
  - **Interpretacija:** **Bliže 1.0 je bolje.** (Raspon je 0-1). Ako je slika mutna, SSIM će pasti drastičnije nego PSNR.

- **CIEDE2000 (ΔE) - Perceptualna razlika boje**
  - **Što mjeri:** Preciznost boja u Lab prostoru, uzimajući u obzir nelinearnost ljudskog vida.
  - **Interpretacija:** **Manje je bolje.** ΔE < 1 znači da je razlika nevidljiva oku. ΔE > 2-3 znači vidljive greške u boji (npr. žuti artefakti).

- **PSNR po kanalima (R, G, B)**
  - **Što mjeri:** Točnost svakog pojedinog kanala boje.
  - **Interpretacija:** **Veće je bolje.** Pomaže detektirati koji kanal najviše pati (često R i B kod Quad Bayera).


## 1. Priprema okruženja

Učitavamo potrebne biblioteke i backend modul koji sadrži:
- Funkcije za generiranje mozaika (`generate_quad_bayer_mosaic`, `generate_nonacell_mosaic`)
- Metode za demozaiciranje (Direct, Virtual Bayer, Super-Resolution)
- Funkcije za evaluaciju kvalitete rekonstrukcije
- Funkcije za prikaz rezultata

In [ ]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import backend  # Naš custom modul s algoritmima

noise_level = 0.02 # Postavi željenu razinu simuliranog šuma na senzoru
print("Sustav spreman.")

---

# Quad Bayer Metode

## 1.1 Quad Bayer Direct
**Slika:** `kodim19.png` (Svjetionik i ograda)

**Zašto ova slika?** Kose linije ograde su savršene za demonstraciju performansi na rubovima.

In [ ]:
img = backend.load_and_prep('kodim19.png')
raw = backend.simulate_noise(backend.generate_quad_bayer_mosaic(img), noise_level)

recon = backend.demosaic_direct_quad_bayer(raw)
res = backend.evaluate_reconstruction(img, recon, "QB Direct")

backend.show_comparison(img, raw, recon, "Quad Bayer Direct - Ograda", mosaic_type="Quad Bayer")
print(f"PSNR: {res['PSNR (Y)']:.2f} dB")
print(f"SSIM: {res['SSIM (Y)']:.4f}")

In [ ]:
# Vizualizacija artefakata (Zoom)
y_z, x_z, size = 320, 250, 80
zoom_orig = img[y_z:y_z+size, x_z:x_z+size]
zoom_recon = recon[y_z:y_z+size, x_z:x_z+size]

fig, ax = plt.subplots(1, 2, figsize=(12, 6))
ax[0].imshow(cv2.cvtColor(zoom_orig, cv2.COLOR_BGR2RGB))
ax[0].set_title("Original - Detalj ograde")
ax[0].axis("off")
ax[1].imshow(cv2.cvtColor(zoom_recon, cv2.COLOR_BGR2RGB))
ax[1].set_title("Rekonstrukcija - Uočite žute rubove")
ax[1].axis("off")
plt.show()

**Komentar:**

**1. Žuti artefakti na ogradi:** Na bijeloj ogradi oko svjetionika jasno je vidljiva pojava lažne žute boje (false color artifacts). Ovo je klasičan primjer color aliasinga specifičnog za Quad Bayer senzore. Budući da su pikseli iste boje grupirani u 2x2 blokove, prostorna rezolucija uzorkovanja boja je smanjena. Na tankim, ponavljajućim uzorcima visoke frekvencije (poput rešetki ograde) koji su blizu Nyquistove frekvencije senzora, algoritam pogrešno interpolira vrijednosti boja. Konkretno, 'curenje' zelene i crvene komponente (koje čine žutu) u područja koja bi trebala biti neutralna (bijela/siva) ukazuje na to da algoritam nije uspio pravilno rekonstruirati plavu komponentu na tim mjestima.

**2. Mekoća slike:** Slika je općenito mekša od originala. To je očekivana posljedica Direct metode koja koristi šire kernele kako bi premostila veće razmake između piksela različitih boja u Quad Bayer rasporedu.

---

## 1.2 Quad Bayer Binning
**Slika:** `kodim23.png` (Papige)

**Zašto ova slika?** Jarke boje i fine teksture perja.

In [ ]:
img = backend.load_and_prep('kodim23.png')
raw = backend.simulate_noise(backend.generate_quad_bayer_mosaic(img), noise_level)

recon = backend.demosaic_virtual_bayer(raw)
res = backend.evaluate_reconstruction(img, recon, "QB Virtual Bayer")

backend.show_comparison(img, raw, recon, "Quad Bayer Virtual Bayer - Papige", mosaic_type="Quad Bayer")
print(f"PSNR: {res['PSNR (Y)']:.2f} dB")
print(f"SSIM: {res['SSIM (Y)']:.4f}")

In [ ]:
# Vizualizacija artefakata (Zoom)
y_z, x_z, size = 150, 350, 100
h, w = img.shape[:2]
y_z = min(y_z, h - size)
x_z = min(x_z, w - size)

zoom_orig = img[y_z:y_z+size, x_z:x_z+size]
zoom_recon = recon[y_z:y_z+size, x_z:x_z+size]

fig, ax = plt.subplots(1, 2, figsize=(12, 6))
ax[0].imshow(cv2.cvtColor(zoom_orig, cv2.COLOR_BGR2RGB))
ax[0].set_title("Original - Detalj")
ax[0].axis("off")
ax[1].imshow(cv2.cvtColor(zoom_recon, cv2.COLOR_BGR2RGB))
ax[1].set_title("Binning - Gubitak ostrine")
ax[1].axis("off")
plt.show()

**Komentar:**

Binning metoda usrednjava vrijednosti piksela unutar 2x2 bloka. To smanjuje šum, ali znatno smanjuje i rezoluciju slike. Kao što se vidi na zoom-u, detalji su zamućeni (perje papiga), ali nema izraženih artefakata u boji (poput žute na ogradi u 1.1) jer ova metoda djeluje kao low-pass filter.

---

## 1.3 Quad Bayer Super Resolution
**Slika:** `kodim01.png` (Zgrada)

**Zašto ova slika?** Ravne linije.

In [ ]:
img = backend.load_and_prep('kodim01.png')
raw = backend.simulate_noise(backend.generate_quad_bayer_mosaic(img), noise_level)

recon = backend.demosaic_super_resolution_quad_bayer(raw)
res = backend.evaluate_reconstruction(img, recon, "QB Super-Res")

backend.show_comparison(img, raw, recon, "Quad Bayer Super-Resolution - Zgrada", mosaic_type="Quad Bayer")
print(f"PSNR: {res['PSNR (Y)']:.2f} dB")
print(f"SSIM: {res['SSIM (Y)']:.4f}")

In [ ]:
# Vizualizacija artefakata (Zoom)
y_z, x_z, size = 300, 400, 80
h, w = img.shape[:2]
y_z = min(y_z, h - size)
x_z = min(x_z, w - size)

zoom_orig = img[y_z:y_z+size, x_z:x_z+size]
zoom_recon = recon[y_z:y_z+size, x_z:x_z+size]

fig, ax = plt.subplots(1, 2, figsize=(12, 6))
ax[0].imshow(cv2.cvtColor(zoom_orig, cv2.COLOR_BGR2RGB))
ax[0].set_title("Original - Tekstura")
ax[0].axis("off")
ax[1].imshow(cv2.cvtColor(zoom_recon, cv2.COLOR_BGR2RGB))
ax[1].set_title("Super Resolution")
ax[1].axis("off")
plt.show()

**Komentar:**

Super Resolution pristup pokušava rekonstruirati detalje koristeći informacije iz svih piksela u bloku, ne samo prosjek. Rezultat bi trebao biti oštriji od Binninga.

---

# Nonacell Metode

## 2.1 Noncacell Direct
**Slika:** `zone_plate.png` (Zone plate uzorak)

**Zašto ova slika?** Koncentrične kružnice su teške za rekonstruirati.

In [ ]:
img = backend.load_and_prep('zone_plate.png')
raw = backend.simulate_noise(backend.generate_nonacell_mosaic(img), noise_level)

recon = backend.demosaic_direct_nonacell(raw)
res = backend.evaluate_reconstruction(img, recon, "Nonacell Direct")

backend.show_comparison(img, raw, recon, "Nonacell Direct - Zone Plate", mosaic_type="Nonacell")
print(f"PSNR: {res['PSNR (Y)']:.2f} dB")
print(f"SSIM: {res['SSIM (Y)']:.4f}")

In [ ]:
# Vizualizacija artefakata (Zoom)
y_z, x_z, size = 256, 256, 128
h, w = img.shape[:2]
y_z = min(y_z, h - size)
x_z = min(x_z, w - size)

zoom_orig = img[y_z:y_z+size, x_z:x_z+size]
zoom_recon = recon[y_z:y_z+size, x_z:x_z+size]

fig, ax = plt.subplots(1, 2, figsize=(12, 6))
ax[0].imshow(cv2.cvtColor(zoom_orig, cv2.COLOR_BGR2RGB))
ax[0].set_title("Original - Centar")
ax[0].axis("off")
ax[1].imshow(cv2.cvtColor(zoom_recon, cv2.COLOR_BGR2RGB))
ax[1].set_title("Direct - Lažne boje")
ax[1].axis("off")
plt.show()

**Komentar:**

Kod Nonacell strukture, Direct metoda se bori s još nižom frekvencijom uzorkovanja boja. Na ovom sintetičkom testu vidimo snaže lažne boje.

---

## 2.2 Nonacell Binning
**Slika:** `kodim04.png` (Djevojka)

**Zašto ova slika?** Portret zahtijeva prirodan izgled kože.

In [ ]:
img = backend.load_and_prep('kodim04.png')
raw = backend.simulate_noise(backend.generate_nonacell_mosaic(img), noise_level)

recon = backend.demosaic_virtual_bayer_nonacell(raw)
res = backend.evaluate_reconstruction(img, recon, "Nonacell Virtual Bayer")

backend.show_comparison(img, raw, recon, "Nonacell Virtual Bayer - Žena sa šeširom", mosaic_type="Nonacell")
print(f"PSNR: {res['PSNR (Y)']:.2f} dB")
print(f"SSIM: {res['SSIM (Y)']:.4f}")

In [ ]:
# Vizualizacija artefakata (Zoom)
y_z, x_z, size = 200, 250, 100
h, w = img.shape[:2]
y_z = min(y_z, h - size)
x_z = min(x_z, w - size)

zoom_orig = img[y_z:y_z+size, x_z:x_z+size]
zoom_recon = recon[y_z:y_z+size, x_z:x_z+size]

fig, ax = plt.subplots(1, 2, figsize=(12, 6))
ax[0].imshow(cv2.cvtColor(zoom_orig, cv2.COLOR_BGR2RGB))
ax[0].set_title("Original - Lice")
ax[0].axis("off")
ax[1].imshow(cv2.cvtColor(zoom_recon, cv2.COLOR_BGR2RGB))
ax[1].set_title("Binning - Izražena mekoća")
ax[1].axis("off")
plt.show()

**Komentar:**

Nonacell Binning smanjuje rezoluciju slike za faktor 3 u svakom smjeru (ukupno 9 piksela u 1). Rezultat je slika vrlo niske rezolucije. Iako su boje točne, fini detalji lica i teksture su potpuno izgubljeni, a slika se čini vrlo izglađenom.

---

## 2.3 Nonacell Super Resolution
**Slika:** `kodim04.png` (Curica s bojama)

**Zašto ova slika?** Portret sadrži miks boja, kože i teksture tkanine.

In [ ]:
img = backend.load_and_prep('kodim15.png')
raw = backend.simulate_noise(backend.generate_nonacell_mosaic(img), noise_level)

recon = backend.demosaic_super_resolution_nonacell(raw)
res = backend.evaluate_reconstruction(img, recon, "Nonacell Super Resolution")

backend.show_comparison(img, raw, recon, "Nonacell Super Resolution - Curica s bojama", mosaic_type="Nonacell")
print(f"PSNR: {res['PSNR (Y)']:.2f} dB")
print(f"SSIM: {res['SSIM (Y)']:.4f}")

In [ ]:
# Vizualizacija artefakata (Zoom)
y_z, x_z, size = 200, 250, 100
h, w = img.shape[:2]
y_z = min(y_z, h - size)
x_z = min(x_z, w - size)

zoom_orig = img[y_z:y_z+size, x_z:x_z+size]
zoom_recon = recon[y_z:y_z+size, x_z:x_z+size]

fig, ax = plt.subplots(1, 2, figsize=(12, 6))
ax[0].imshow(cv2.cvtColor(zoom_orig, cv2.COLOR_BGR2RGB))
ax[0].set_title("Original - Tkanina")
ax[0].axis("off")
ax[1].imshow(cv2.cvtColor(zoom_recon, cv2.COLOR_BGR2RGB))
ax[1].set_title("Binning - Gubitak detalja")
ax[1].axis("off")
plt.show()

**Komentar:**

Nonacell Super Resolution nam je izgubio detalje u slici i vidimo tračak pojavljivanja lažnih boja.

---


# 4. Grafički prikaz rezultata

Usporedba performansi svih metoda kroz testne slike. Grafovi prikazuju PSNR, SSIM i prosječne vrijednosti za svaku metodu.

In [ ]:
# Vizualizacija rezultata svih metoda na cijelom testnom skupu podataka

# noise_level = 0.1 # Odkomentiraj ovu liniju koda za ručnu promjenu razine šuma

df, avg_df = backend.plot_results_comparison(noise_level)

---
# 5. Interaktivni Demo

Ovdje možete uploadati vlastitu sliku i testirati metode rekonstrukcije za individualne metode ili pak za sve metode odjednom.

**Upute:** Kliknite na "Upload" te odaberite sliku s računala.

In [ ]:
backend.create_interactive_demo()